# HR Analytics Dashboard: Employee Attrition

This project explores the contributing factors to employee attrition based on real-world data adopted from Kaggle. The goal is to answer the following questions: 

1. What is the correlation between attrition and demographics, compensation, work style, and employee attitude? 
2. What is the correlation among the dimensions, i.e. demographics, compensation, work style, and employee attitude? 
3. What is most predictive of attrition among these variables? 
4. What are some actionable suggestions for reducing attrition? 

This project is the first research practice of FireHua Consulting LLC. 

*📥 To run this notebook, download the dataset [here](./Cleaned_Employee_Data.xlsx) and make sure it is in the same folder as this notebook.*

In [1]:
# Import packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

## Dataset Overview

The original dataset contains data in various domains, where some examples include employee IDs and hourly rates. After careful review of data, those of interest are organized into a separate worksheet within the original Excel file. 

Key variables of interest include: 
- **Attrition**: *Target variable*, encoded to binary numbers, "1" indicating "Yes" and "0" indicating "No" for attrition. 
- **Demographics**: Age, Total Working Years, Education
- **Work Style**: Over Time, Work Life Balance
- **Compensation**: Monthly Income, Percentage Salary Hike, Stock Option Level, Years Since Last Promotion
- **Employee attitude**: Environment Satisfaction, Job Satisfaction, Relationship Satisfaction, Total Satisfaction Score

The exact definitions, units, and coding explanations are included in the worksheet named *"Variables"* within the Excel file. 

In [2]:
# Load the cleaned dataset from the uploaded Excel file
file_path = "Cleaned_Employee_Data.xlsx"
df = pd.read_excel(file_path, sheet_name="Cleaned Data")

# Show the first few rows of the dataset to confirm successful load
print(df.head())

   Attrition  Age  Total Working Years  Education  Over Time  \
0          1   41                    8          2          1   
1          0   49                   10          1          0   
2          1   37                    7          2          1   
3          0   33                    8          4          1   
4          0   27                    6          1          0   

   Work Life Balance  Monthly Income  Percent Salary Hike  Stock Option Level  \
0                  1            5993                   11                   0   
1                  3            5130                   23                   1   
2                  3            2090                   15                   0   
3                  3            2909                   11                   0   
4                  3            3468                   12                   1   

   Years Since Last Promotion  Environment Satisfaction  Job Satisfaction  \
0                           0                      

## Correlational Analysis

### Create Functions

To answer the first question (*What is the correlation between attrition and demographics, compensation, work style, and employee attitude?*), we need to run a correlational analysis that provides both the correlation coefficients and p-value (set to *p<0.05*) indicating their statistical significance. The correlational analysis used in this project is **Pearson coefficient**. Upon calculation, we also need to store them into the a dataset for clear presentation. Therefore, the function created below serves two purposes: 

1. Run Pearson correlational analysis for each variable. 
2. Store results in one place. 

In [3]:
# Function for running and storing each correlation comparison 
def get_correlation_table(df, target, predictors, significance_level=0.05):
    results = []

    for col in predictors:
        if col == target:
            continue
        try:
            r, p = pearsonr(df[target], df[col])
            results.append({
                "Variable": col,
                "Correlation (r)": round(r, 2),
                "P-value": round(p, 5),
                "Significant (p < 0.05)": "Yes" if p < significance_level else "No"
            })
        except Exception as e:
            results.append({
                "Variable": col,
                "Correlation (r)": "ERROR",
                "P-value": str(e),
                "Significant (p < 0.05)": "No"
            })

    return pd.DataFrame(results)

### Run Function and Get Results

With the function ready, we can start the actual data analysis. It includes three parts: 
1. Define variables of interest 
2. Run correlational analysis
3. Store results into a new Excel file 

In [ ]:
## Define variables of interest 
# Attrition vs Demographics
attr_demographics = [
    "Attrition", 
    "Age", 
    "Total Working Years", 
    "Education"
]
# Attrition vs Work Style
attr_work_style = [
    "Over Time", 
    "Work Life Balance"
]
# Attrition vs Compensation
attr_compensation = [
    "Monthly Income", 
    "Percent Salary Hike", 
    "Stock Option Level", 
    "Years Since Last Promotion"
]
# Attrition vs Employee Attitude
attr_satisfaction = [
    "Environment Satisfaction", 
    "Job Satisfaction", 
    "Relationship Satisfaction", 
    "Total Satisfaction Score"
]

## Run correlational analysis
attr_demo_corr = get_correlation_table(df, target="Attrition", predictors=attr_demographics)
attr_style_corr = get_correlation_table(df, target="Attrition", predictors=attr_work_style)
attr_comp_corr = get_correlation_table(df, target="Attrition", predictors=attr_compensation)
attr_sat_corr = get_correlation_table(df, target="Attrition", predictors=attr_satisfaction)

## Store results 
with pd.ExcelWriter("correlation_results.xlsx", 
  mode="a", 
  engine="openpyxl") as writer:
    attr_demo_corr.to_excel(writer, sheet_name="Attr_Demo")
    attr_style_corr.to_excel(writer, sheet_name="Attr_Work_Style")
    attr_comp_corr.to_excel(writer, sheet_name="Attr_Comp")
    attr_sat_corr.to_excel(writer, sheet_name="Attr_Sat")

PermissionError: [Errno 13] Permission denied: 'correlation_results.xlsx'

### Excel Visualization
![Correlation Bar Chart](Attr_Var_Corr.png)

**Observations**: 
- 9 out of the 13 variables have statistically significant (*p < 0.05*) correlation with attrition, while only 6 are considered meaningful with a threshold of *r = .1* for meaningful correlation. The variables that are both significant and meaningful include **Over Time**, **Total Working Years**, **Monthly Income**, **Age**, **Total Satisfaction Score**, and **Stock Option Level**. 
- Out of the variables that are significant and meaningful, only **Over Time** is positively correlated with attrition, with a correlation of *r = .25*. While significant, none of the negative correlations are as high as **Over Time**, with the highest being **Total Working Years** (*r = -.17*). 
- Although **Environment Satisfaction**, **Job Satisfaction**, and **Relationship Satisfaction** appear either insignificant or not meaningful, the composite score **Total Satisfaction Score** is negatively correlated with attrition with a correlational coefficient of *r = -.15*. 
- 2 variables under the dimension of **Compensation** appear to be significant and meaningful, including **Total Working Years** (*r = -.17*) and **Monthly Income** (*r = -.16*). 

**Takeaways**: 
- **Over Time** likely has a relatively strong impact on attrition in this company and needs further investigation for causational effect. 
- **Compensation** seems to have a contributing impact on attrition, despite its smaller correlational coefficient compared to **Over Time**
- Employee attitude is complicated and should be considered as a whole. The relationship between **Total Satisfaction Score** and its sub-variables is worth digging into. 

## Next Steps...

Now that we have identified the variables that have significant and meaningful correlations with attrition, we need to investigate the causational effects. Therefore, the next step is to conduct *regression analysis* to test the predictive effects of these variables. 
